# 인기 급상승 유튜버 랭크 요인 분석
> 인플루언서 랭킹 사이트 크롤링

1. 유튜버 순위 정보 크롤링 --> Done
    * 녹스인플루언서에서 '인기 급상승 순위' 크롤링
        * URL : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-all-all-youtuber-sorted-by-growth-weekly
    * 유튜브 카테고리를 변경하면서 카테고리별 순위 페이지에서 유튜버 개별 페이지 URL을 크롤링.
    * 유튜버 개별 페이지 크롤링으로 전달


2. 유튜버 개별 페이지 크롤링  --> Done
    * 녹스인플루언서 개별 페이지에서 '30일간 유튜브 통계', '유튜버 채널 정보', 유튜브 채널 페이지에서 설명 크롤링
    * URL : https://kr.noxinfluencer.com/ + 유튜버 채널 URL --> 녹스인플루언서 개별 페이지
    * URL : https://www.youtube.com/ + 유튜버 채널 URL + '/about' --> 유튜브 채널 > 정보 페이지에서 채널 설명 크롤링
    * 크롤링 한 정보를 데이터프레임으로 가공


3. Concat 한 DataFrame 탐색 --> 진행중
    * 인기 급상승 유튜버 30일 평균 동영상 업로드 수
    * 랭크된 유튜버의 30일간 구독자 증감율


In [345]:
# Basci pakage import

import time
import json
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Selenium import

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings(action='ignore')

# BeautifulSoup import

from bs4 import BeautifulSoup

In [348]:
# 크롬 드라이버 객체 생성 함수

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [346]:
# 셀레니움 설치
# !pip install selenium --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [347]:
# 크롬드라이버 관리 툴 설치
# !pip install webdriver-manager --trusted-host pypi.org --trusted-host files.pythonhosted.org

## 순위에 랭크된 유튜버 개별 페이지 URL 가져오기 --> Done
- 교육 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-education-youtuber-sorted-by-growth-weekly
- 노하우 스타일 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-howto%20%26%20style-youtuber-sorted-by-growth-weekly
- 스포츠 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-sports-youtuber-sorted-by-growth-weekly
- 여행 이벤트 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-travel%20%26%20events-youtuber-sorted-by-growth-weekly
- 과학 기술 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-science%20%26%20technology-youtuber-sorted-by-growth-weekly
- 영화 애니메이션 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-film%20%26%20animation-youtuber-sorted-by-growth-weekly
- 엔터테인먼트 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-entertainment-youtuber-sorted-by-growth-weekly
- 음악 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-music-youtuber-sorted-by-growth-weekly
- 인물 블로그 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-people%20%26%20blogs-youtuber-sorted-by-growth-weekly
- 동물 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-pets%20%26%20animals-youtuber-sorted-by-growth-weekly
- 자동차 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-autos%20%26%20vehicles-youtuber-sorted-by-growth-weekly
- 게임 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-gaming-youtuber-sorted-by-growth-weekly
- 코미디 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-comedy-youtuber-sorted-by-growth-weekly
- 뉴스 정치 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-news%20%26%20politics-youtuber-sorted-by-growth-weekly
- 봉사활동 : https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-nonprofits%20%26%20activism-youtuber-sorted-by-growth-weekly

1) 녹스인플루언서 사이트에서 '인기 급상승 순위' 카테고리 선택한다. --> Done

    - 카테고리 선택 --> 1: 교육, 2: 노하우 스타일, 3: 스포츠, 4: 여행 이벤트, 5: 과학 기술, 6: 영화 애니메이션, 7: 엔터테인먼트, 8: 음악, 9: 인물 블로그, 10: 동물, 11: 자동차, 12: 게임, 13: 코미디, 14: 뉴스 정치, 15: 봉사활동

2) 선택한 카테고리 순위를 크롤링 -> 개별 페이지 URL을 추출한다. --> Done

3) 추출한 URL을 개별 페이지 크롤링에 전달한다. --> Done


In [411]:
# 크롤링 정보를 저장할 빈 DataFrame 생성
final_df = pd.DataFrame(columns=['유튜버', '유튜브가입일', '구독자', '조회수', '동영상', '누적동영상수', '유튜브 수익 예측', '라이브 수입', '정보획득일'])


In [412]:
# 카테고리 선택

print('''
카테고리 목록
===================
1: 교육
2: 노하우 스타일
3: 스포츠
4: 여행 이벤트
5: 과학 기술
6: 영화 애니메이션
7: 엔터테인먼트
8: 음악
9: 인물 블로그
10: 동물
11: 자동차
12: 게임
13: 코미디
14: 뉴스 정치
15: 봉사활동
===================
''')
cat_url = ['https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-education-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-howto%20%26%20style-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-sports-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-travel%20%26%20events-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-science%20%26%20technology-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-film%20%26%20animation-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-entertainment-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-music-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-people%20%26%20blogs-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-pets%20%26%20animals-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-autos%20%26%20vehicles-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-gaming-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-comedy-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-news%20%26%20politics-youtuber-sorted-by-growth-weekly',
'https://kr.noxinfluencer.com/youtube-channel-rank/top-100-kr-nonprofits%20%26%20activism-youtuber-sorted-by-growth-weekly']

category = ['교육', '노하우 스타일', '스포츠', '여행 이벤트', '과학 기술', '영화 애니메이션', '엔터테인먼트', '음악', '인물 블로그', '동물', 
'자동차', '게임', '코미디', '뉴스 정치', '봉사활동']

menu_num = int(input('크롤링 할 유튜브 카테고리 번호를 입력하세요.'))

print(f'{category[menu_num-1]} 카테고리 크롤링을 시작합니다.')


카테고리 목록
1: 교육
2: 노하우 스타일
3: 스포츠
4: 여행 이벤트
5: 과학 기술
6: 영화 애니메이션
7: 엔터테인먼트
8: 음악
9: 인물 블로그
10: 동물
11: 자동차
12: 게임
13: 코미디
14: 뉴스 정치
15: 봉사활동

크롤링 할 유튜브 카테고리 번호를 입력하세요.1
교육 카테고리 크롤링을 시작합니다.


In [413]:
# 셀레니움 사용을 위해 크롬 드라이버 생성한다.

driver = set_chrome_driver()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\rabie\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [414]:
# Request & Soup 객체 생성
resp = requests.get(cat_url[menu_num-1])
html = BeautifulSoup(resp.text, 'html.parser')

# 개별_페이지_크롤링(page_url), 크롤링 갯수 3개로 한정

for page in html.select('a.clearfix')[:3]:
    page_url = page.attrs['href']
    url = nox_site + page_url
    print(f'{url} 페이지를 크롤링 합니다.')
    final_df = pd.concat([final_df, one_youtube_crawl(url)])
    

https://kr.noxinfluencer.com/youtube/channel/UClO3Y9xjoWK7sDD8FuqbtTQ 페이지를 크롤링 합니다.
https://kr.noxinfluencer.com/youtube/channel/UCKaEKEiswhc9gYL-cwrC49w 페이지를 크롤링 합니다.
https://kr.noxinfluencer.com/youtube/channel/UCOXr057Ke81U9BNpsulJA5w 페이지를 크롤링 합니다.


## 셀레니움을 이용, 개별 페이지 크롤링 --> Done

순위 정보에서 얻은 URL을 이용
셀레니움을 이용해서 유튜버 개별 페이지에 있는 '유튜브 채널 통계표'를 크롤링
유튜버 개별 데이터프레임으로 만든다.

In [410]:
def one_youtube_crawl(url):

    # 셀레니움으로 유튜버 개별 페이지 URL로 이동
    driver.get(url)


    # 유튜브 채널 통계표 '30일' 클릭해서 정보를 불러온다.
    time.sleep(6)
    driver.find_element(by='xpath', value='//*[@id="tab-channel"]/div[2]/div[1]/div/span[3]').click()


    # response 얻고 BeautifulSoup 객체로 변환
    time.sleep(3)
    resp = driver.page_source
    html = BeautifulSoup(resp, 'html.parser')


    # 유튜버 이름, 유튜브 가입일, 누적동영상 갯수, 정보획득일자 를 가져온다.

    username = html.select('span.username')[0].text  # 유튜버 이름
    start_date = html.select('p.pull-right.item-value')[0].text.strip() # 유튜브 가입일
    movie_count_cum = html.select('span.strong')[3].text.strip() # 정보획득일 현재까지 누적된 동영상 갯수
    info_date = html.select('span.refresh-text')[0].text.split()[2] + ' ' + html.select('span.refresh-text')[0].text.split()[3] # 정보를 크롤링한 날짜


    # 유튜브 통계표를 긁어서 데이터프레임에 저장한다.

    data_list = [data.text.strip() for data in html.select('table.tabla-ui-content tbody tr td')]
    data = np.array(data_list).reshape(-1, 6)
    youtube_stat = pd.DataFrame(data, columns=column)


    # 데이터프레임에 컬럼을 추가. 컬럼 : '유튜버', '유튜브 가입일', '누적동영상수', '정보획득일'

    youtube_stat['유튜버'] = username
    youtube_stat['유튜브가입일'] = start_date
    youtube_stat['누적동영상수'] = movie_count_cum
    youtube_stat['정보획득일'] = info_date


    # 날짜 컬럼을 인덱스로 변경

    youtube_stat.set_index('날짜', inplace=True)


    # 구독자 - 증감량, 조회수-증감량을 분리 후 1단위로 변환하여 구독자, 조회수에 저장한다.

    youtube_stat['구독자'] = youtube_stat[youtube_stat['구독자'].str.contains('만')==True]['구독자'].apply(lambda x: str(int(float(x.split(sep='만')[0])*10000)))
    youtube_stat['조회수'] = youtube_stat[youtube_stat['조회수'].str.contains('만')==True]['조회수'].apply(lambda x: str(int(float(x.split(sep='만')[0])*10000)))


    # '동영상' 컬럼에 '+', '--'를 없앤다.

    youtube_stat['동영상'] = youtube_stat['동영상'].apply(lambda x: x.replace('+', ''))
    youtube_stat['동영상'] = youtube_stat['동영상'].apply(lambda x: x.replace('--', '0'))


    # '구독자', '조회수', '동영상', '누적동영상수' 컬럼을 int로 타입캐스트

    youtube_stat['구독자'] = youtube_stat['구독자'].astype(int)
    youtube_stat['조회수'] = youtube_stat['조회수'].astype(int)
    youtube_stat['동영상'] = youtube_stat['동영상'].astype(int)
    youtube_stat['누적동영상수'] = youtube_stat['누적동영상수'].astype(int)


    # 컬럼 순서 조정, 개별 페이지 크롤링 완료

    youtube_stat = youtube_stat[['유튜버', '유튜브가입일', '구독자', '조회수', '동영상', '누적동영상수', '유튜브 수익 예측', '라이브 수입', '정보획득일']]


    # (나중에 필요하면)'정보획득일' 컬럼을 datetime 으로 타입캐스트 


    # youtube 에서 직접 채널소개글 가져오기 (연락처 정보를 찾아내자.)

    url_yt = page_url.replace('/youtube', 'https://www.youtube.com') + '/about'

    driver.get(url_yt)
    resp_yt = driver.page_source
    html_yt = BeautifulSoup(resp_yt, 'html.parser')
    user_description = html_yt.select('yt-formatted-string#description')[0].text.replace('\n', '').replace('  ', '') # 유튜버 소개

    # 데이터프레임에 '채널설명' 컬럼 추가
    youtube_stat['채널설명'] = user_description
    
    return youtube_stat


## Concat 완료한 DataFrame 탐색 --> 진행중

    - 인기 급상승 유튜버 30일 평균 동영상 업로드 수
    - 랭크된 유튜버의 30일간 구독자 증감율

In [415]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 2022-05-29 to 2022-06-26
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   유튜버        87 non-null     object
 1   유튜브가입일     87 non-null     object
 2   구독자        87 non-null     object
 3   조회수        87 non-null     object
 4   동영상        87 non-null     object
 5   누적동영상수     87 non-null     object
 6   유튜브 수익 예측  87 non-null     object
 7   라이브 수입     87 non-null     object
 8   정보획득일      87 non-null     object
 9   채널설명       87 non-null     object
dtypes: object(10)
memory usage: 7.5+ KB


In [418]:
final_df.head()

,유튜버,유튜브가입일,구독자,조회수,동영상,누적동영상수,유튜브 수익 예측,라이브 수입,정보획득일,채널설명
2022-05-29,한국가스공사,2016-01-06,10100,5226700,0,245,--,--,2022-06-26 23:17:23,더 좋은 세상을 만드는 ㄱㅅㄱㅅ (가스공사)
2022-05-30,한국가스공사,2016-01-06,10100,5227300,1,245,￦ 1159 - ￦ 2016,--,2022-06-26 23:17:23,더 좋은 세상을 만드는 ㄱㅅㄱㅅ (가스공사)
2022-05-31,한국가스공사,2016-01-06,10100,5228000,0,245,￦ 1493 - ￦ 2596,--,2022-06-26 23:17:23,더 좋은 세상을 만드는 ㄱㅅㄱㅅ (가스공사)
2022-06-01,한국가스공사,2016-01-06,10100,5228500,0,245,￦ 942.95 - ￦ 1640,--,2022-06-26 23:17:23,더 좋은 세상을 만드는 ㄱㅅㄱㅅ (가스공사)
2022-06-02,한국가스공사,2016-01-06,10200,5229100,0,245,￦ 1122 - ￦ 1951,--,2022-06-26 23:17:23,더 좋은 세상을 만드는 ㄱㅅㄱㅅ (가스공사)
